In [1]:
!pip install mediapipe opencv-python numpy

In [21]:
import cv2
import mediapipe as mp
import numpy as np
from google.colab import files
from IPython.display import Video

In [32]:
uploaded = files.upload()

Saving bicep_curl.mp4 to bicep_curl (1).mp4


In [33]:
input_video_path="bicep_curl (1).mp4"
output_video_path="Analysed_bicep_curl (1).mp4"

In [34]:
#Angle function
def calculate_angle(a,b,c):
  a,b,c=np.array(a),np.array(b),np.array(c)
  ba=a-b
  bc=c-b
  cosine=np.dot(ba,bc)/(np.linalg.norm(ba)*np.linalg.norm(bc)+1e-6)
  cosine=np.clip(cosine,-1.0,1.0)
  return np.degrees(np.arccos(cosine))

In [35]:
from collections import deque
class AngleSmoother:
  def __init__(self,window_size=5):
    self.values=deque(maxlen=window_size)

  def add(self,value):
    self.values.append(value)
    return np.mean(self.values)

In [41]:
def analyse_bicep_curl(input_path, output_path):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Unable to open video")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    pose = mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)

    elbow_smooth = AngleSmoother()
    back_smooth = AngleSmoother()

    rep_count = 0
    stage = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        feedback = "No person detected"
        elbow_angle = None
        back_angle = None

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            # Important joints
            def pt(p): return [lm[p].x * width, lm[p].y * height]

            shoulder = pt(mp_pose.PoseLandmark.RIGHT_SHOULDER.value)
            elbow    = pt(mp_pose.PoseLandmark.RIGHT_ELBOW.value)
            wrist    = pt(mp_pose.PoseLandmark.RIGHT_WRIST.value)
            hip      = pt(mp_pose.PoseLandmark.RIGHT_HIP.value)
            knee     = pt(mp_pose.PoseLandmark.RIGHT_KNEE.value)

            #Form evaluation logic
            elbow_raw = calculate_angle(shoulder, elbow, wrist)
            elbow_angle = elbow_smooth.add(elbow_raw)

            back_raw = calculate_angle(shoulder, hip, knee)
            back_angle = back_smooth.add(back_raw)

            # REP COUNTING
            if elbow_angle > 150:
                stage = "down"
            if elbow_angle < 80 and stage == "down":
                stage = "up"
                rep_count += 1

            warnings = []

            # Rule 1: elbow angle
            if elbow_angle < 40:
                warnings.append("elbow and Curl flexed too tight")
            if elbow_angle > 170:
                warnings.append("Over-flexion elbow")

            # Rule 2: back posture(Back/Hip Stability)
            if back_angle < 160:
                warnings.append("Keep hip stable")

            #Rule 3: Wrist Alginment
            wrist_difference=abs(wrist[1]-elbow[1])
            if wrist_difference>40:
              warnings.append("Keeep the wrist straight")

            # Final feedback
            if warnings:
                feedback = warnings[0]
            else:
                feedback = "Good form"

            # Drawing  pose lines
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Frame-wise feedback
            cv2.putText(frame, f"Elbow: {int(elbow_angle)}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(frame, f"Back: {int(back_angle)}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(frame, f"Reps: {rep_count}", (10, 90),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
            cv2.putText(frame, feedback, (10, height-20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print("Processing complete:", output_path)

In [42]:
analyse_bicep_curl(input_video_path, output_video_path)

Processing complete: Analysed_bicep_curl (1).mp4


In [43]:
!ffmpeg -i "{output_video_path}" -vcodec libx264 -acodec aac "final_output.mp4" -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [44]:
print("uploaded File:",input_video_path)

uploaded File: bicep_curl (1).mp4


In [45]:
Video("final_output.mp4", embed=True)